In [1]:
import pandas as pd
import json
import os

df = pd.read_csv("../data/results/all_dependencies_updated.csv")

df["final_rating"]


def transform_result_file(results_file: str):

    parts = results_file.split("/")    
    config = parts[3]
    llm_name = parts[-1].split("_")[-1].split(".json")[0]


    with open(results_file, "r", encoding="utf-8") as src:
        data = json.load(src)

    for entry in data:
        gt = df.iloc[entry["index"]]

        if "responses" in entry:
            response = entry["responses"][0]
        else:
            response = entry["response"]
        
        try:
           
            response_dict = json.loads(response, strict=False)
            entry["response_dict"] = response_dict
            entry["reasoning"] = response_dict["rationale"]
            entry["uncertainty"] = response_dict["uncertainty"]
    

            if gt["final_rating"] == response_dict["isDependency"]:
                if gt["final_rating"]:
                    entry["correct"] = "TP"
                else:
                    entry["correct"] = "TN"
            else:
                if gt["final_rating"]:
                    entry["correct"] = "FN"
                else:
                    entry["correct"] = "FP"

        except:
            entry["response_dict"] = response
            entry["reasoning"] = None
            entry["uncertainty"] = None
            entry["correct"] = None

        entry["final_category"] = gt["final_category"]
        entry["sub_category"] = gt["sub_category"]
        entry["llm"] = llm_name
        entry["config"] = config


    with open(results_file, "w", encoding="utf-8") as dest:
        json.dump(data, dest, indent=2)

In [2]:
results_path = "../data/results/"
folder_paths = [results_path + x + "/" for x in os.listdir(results_path) if ".csv" not in x]

files = []

for path in folder_paths:
    for result_file in [path + x for x in os.listdir(path) if ".json" in x]:
        files.append(result_file)

In [3]:
for file in files:
    print(file)
    transform_result_file(results_file=file)

../data/results/config4/all_dependencies_all_gpt-4o-2024-05-13.json
../data/results/config4/all_dependencies_all_llama3:70b.json
../data/results/config4/all_dependencies_all_gpt-3.5-turbo-0125.json
../data/results/config4/all_dependencies_all_llama3:8b.json
../data/results/without/all_dependencies_without_gpt-3.5-turbo-0125.json
../data/results/without/all_dependencies_without_llama3:8b.json
../data/results/without/all_dependencies_without_llama3:70b.json
../data/results/without/all_dependencies_without_gpt-4o-2024-05-13.json
../data/results/config2/all_dependencies_all_gpt-4o-2024-05-13.json
../data/results/config2/all_dependencies_all_llama3:70b.json
../data/results/config2/all_dependencies_all_gpt-3.5-turbo-0125.json
../data/results/config2/all_dependencies_all_llama3:8b.json
../data/results/config3/all_dependencies_all_gpt-4o-2024-05-13.json
../data/results/config3/all_dependencies_all_llama3:70b.json
../data/results/config3/all_dependencies_all_gpt-3.5-turbo-0125.json
../data/resu

In [4]:
df = pd.DataFrame()

for file in files:
    with open(file, "r", encoding="utf-8") as src:
        data = json.load(src)
    
    df = pd.concat([df, pd.DataFrame(data)], axis=0, ignore_index=True)

columns = ["llm", "config", "uncertainty", "correct", "reasoning", "context_str", "project", "option_name", "option_type", "option_value","option_technology", 
           "option_file", "dependent_option_name", "dependent_option_type", "dependent_option_value",
           "dependent_option_technology", "dependent_option_file", "final_category", "sub_category"]

json_df = pd.json_normalize(df["dependency"])

df = pd.concat([df.drop("dependency", axis=1), json_df], axis=1)

df = df[columns]
df_2 = df
df = df[df["correct"].isin(["FP", "FN"])]

df = df[df["config"].isin(["without", "config2"])]

df = df[df["llm"].isin(['gpt-4o-2024-05-13', 'llama3:70b', 'llama3:8b', 'gpt-3.5-turbo-0125'])]

df.to_csv("../data/analysis/failures.csv")

In [5]:
len(df)

1203

In [6]:
df["uncertainty"].describe()

count    1203.000000
mean        7.711554
std         1.095639
min         2.000000
25%         7.000000
50%         8.000000
75%         8.000000
max        10.000000
Name: uncertainty, dtype: float64

In [7]:
from scipy.stats import ttest_ind

df_2 = df_2[df_2["correct"].isin(["TP", "TN"])]
df_2 = df_2[df_2["config"].isin(["without", "config2"])]
df_2 = df_2[df_2["llm"].isin(['gpt-4o-2024-05-13', 'llama3:70b', 'llama3:8b', 'gpt-3.5-turbo-0125'])]

df_2["uncertainty"].describe()

#ttest_ind(df["uncertainty"], df_2["uncertainty"])

count    2678.000000
mean        7.964899
std         1.262176
min         0.000000
25%         8.000000
50%         8.000000
75%         9.000000
max        10.000000
Name: uncertainty, dtype: float64

In [11]:
import pandas as pd


df = pd.read_csv("../data/analysis/failures.csv")

#df = pd.read_csv("../data/results/all_dependencies.csv")


dup_columns = ["option_value", "option_name", "option_file", "dependent_option_name", "dependent_option_value", "dependent_option_file", "llm", "config"]

duplicates = df[df.duplicated(subset=dup_columns)]

#for index, row in duplicates.iterrows():
#    print(row["link_str"])

print(len(df))

df = df.drop_duplicates(subset=dup_columns, keep="first")


print(len(df))

duplicates.index

1203
1203


Index([], dtype='int64')

In [10]:
df = pd.read_csv("../data/results/all_dependencies_updated.csv")


dup_columns = ["option_value", "option_name", "option_file", "dependent_option_name", "dependent_option_value", "dependent_option_file"]

duplicates = df[df.duplicated(subset=dup_columns)]

for index, row in duplicates.iterrows():
    print(row["link_str"])

print(len(df))

df = df.drop_duplicates(subset=dup_columns, keep="first")


print(len(df))

duplicates.index

500
500


Index([], dtype='int64')

In [31]:
df = pd.read_csv("../data/analysis/failures_annotated.csv")

print(len(df))

dup_columns = ["option_value", "option_name", "option_file", "dependent_option_name", "dependent_option_value", "dependent_option_file", "llm", "config"]

duplicates = df[df.duplicated(subset=dup_columns)]

df = df.drop_duplicates(subset=dup_columns, keep="first")

print(len(df))

1203
1203
